In [1]:
!pip install accelerate quanto transformers peft

  Using cached accelerate-0.29.2-py3-none-any.whl (297 kB)
  Using cached quanto-0.1.0-py3-none-any.whl (41 kB)
  Using cached peft-0.10.0-py3-none-any.whl (199 kB)
  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)


In [2]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
  Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl (119.8 MB)


In [3]:
import pandas as pd
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from abc import ABC, abstractmethod

In [22]:
class PromptGenerator():
    def __init__(self,
                     prompt: str = """Ti forniró delle triple composte da un oggetto,  una relazione e quattro possibili risposte. Genera una domanda a scelta multipla.
Esempio: oggetto1 =  libro '1984'  relazione = autore risposta1= Huxley risposta2 = Orwell risposta3 = Hemingway risposta4 = Bradbury
Domanda: Chi ha scritto il libro '1984'?
A) Huxley
B) Orwell
C) Hemingway
D) Bradbury

 oggetto1 = {object_target} relazione={relationship} risposta1= {answer1} risposta2 = {answer2} risposta3 =  {answer3} risposta4 = {answer4}"""
                     ):
      self.prompt = prompt
      pass

    def generate(self,
                 object_target,
                 relationship,
                 answer1,
                 answer2,
                 answer3,
                 answer4
                ):
        """
        Generate text based on the input prompt.

        Args:
        object_target (str): the subject of the question
        relationship (str): the relationshio of the question
        answer1 (str): one of the answer
        answer2 (str): one of the answer
        answer3 (str): one of the answer
        answer4 (str): one of the answer

        Returns:
        str: The final prompt of the LLM
        """
        return self.prompt.format(
            object_target=object_target,
            relationship=relationship,
            answer1=answer1,
            answer2=answer2,
            answer3=answer3,
            answer4=answer4
            )

In [7]:
class TextGenerator(ABC):
    """
    Abstract base class for text generation models.
    """

    @abstractmethod
    def generate(self, prompt: str, max_length: int = 256) -> str:
        """
        Generate text based on the input prompt.

        Args:
        prompt (str): The input text prompt to generate text from.
        max_length (int): The maximum length of the generated text.

        Returns:
        str: The generated text.
        """
        pass

In [8]:
class FaunoModel(TextGenerator):
    def __init__(self, device: str = "cpu"):
        self.tokenizer = LlamaTokenizer.from_pretrained("baffo32/decapoda-research-llama-7B-hf")
        self.model = LlamaForCausalLM.from_pretrained(
            "baffo32/decapoda-research-llama-7B-hf",
            load_in_8bit=True,
            device_map=device
        )
        self.model = PeftModel.from_pretrained(self.model, "andreabac3/Open_Fauno-Italian-LLM-7bB")
        self.model.eval()

    def generate(self, question, max_length=256):
        prompt = f"The conversation between human and AI assistant.\n[|Human|] {question}.\n[|AI|] "
        inputs = self.tokenizer(prompt, return_tensors="pt")
        input_ids = inputs["input_ids"].cuda()
        generation_output = self.model.generate(
            input_ids=input_ids,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256
        )
        output = self.tokenizer.decode(generation_output.sequences[0]).split("[|AI|]")[1]
        return output

In [9]:
class LLaMantinoModel(TextGenerator):
    def __init__(self,
                 device: str = "cuda",
                 model_id: str = "swap-uniba/LLaMAntino-2-7b-hf-dolly-ITA",
                 quantization : str = "float8"
                ):
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.tokenizer.add_special_tokens({"pad_token":"<unk>"})
        self.tokenizer.chat_template =   "{% set ns = namespace(i=0) %}" \
                                    "{% for message in messages %}" \
                                        "{% if message['role'] == 'user' and ns.i == 0 %}" \
                                               "{{ bos_token +' [INST] <<SYS>>\n' }}" \
                                               "{{ 'Sei un assistente disponibile, rispettoso e onesto di nome Llamantino. ' }}" \
                                               "{{ 'Rispondi sempre nel modo più utile possibile, pur essendo sicuro. ' }}" \
                                               "{{ 'Le risposte non devono includere contenuti dannosi, non etici, razzisti, sessisti, tossici, pericolosi o illegali. ' }}" \
                                               "{{ 'Assicurati che le tue risposte siano socialmente imparziali e positive. ' }}" \
                                               "{{ 'Se una domanda non ha senso o non è coerente con i fatti, spiegane il motivo invece di rispondere in modo non corretto. ' }}" \
                                               "{{ 'Se non conosci la risposta a una domanda, non condividere informazioni false.\n' }}" \
                                               "{{ '<</SYS>>\n\n' }}" \
                                               "{{ message['content'] + ' [/INST]' }}" \
                                        "{% elif message['role'] == 'user' and ns.i != 0 %} " \
                                            "{{ bos_token + ' [INST] ' + message['content'] + ' [/INST]' }}" \
                                        "{% elif message['role'] == 'assistant' %}" \
                                            "{{ ' '  + message['content'] + ' ' + eos_token + ' ' }}" \
                                        "{% endif %}" \
                                        "{% set ns.i = ns.i+1 %}" \
                                    "{% endfor %}"

        #quantization_config = QuantoConfig(weights=quantization)
        self.model = AutoModelForCausalLM.from_pretrained(model_id,
                                                          device_map=device,
                                                          load_in_8bit=True,
                                                          #quantization_config=quantization_config
                                                          )
        self.model.eval()

        self.pipe = pipeline(model=self.model,
            device_map="balanced",
            tokenizer=self.tokenizer,
            return_full_text=False,  # langchain expects the full text
            task='text-generation',
            max_new_tokens=512,  # max number of tokens to generate in the output
            temperature=0.7 #temperature
        )

    def generate(self, prompt, max_length=256):
        messages = [{"role": "user", "content": prompt}]
        text = self.tokenizer.apply_chat_template(messages, tokenize=False)

        sequences = self.pipe(text)
        output = ""
        for seq in sequences:
            output += output + seq['generated_text']
        return output

## Data Loading

In [10]:
# Download the datasets
!wget https://www.dropbox.com/s/6sbhm0rwo4l73jq/wikidata5m_transductive.tar.gz?dl=1 -O wikidata5m_transductive.tar.gz
!wget https://www.dropbox.com/s/7jp4ib8zo3i6m10/wikidata5m_text.txt.gz?dl=1 -O wikidata5m_text.txt.gz
!wget https://www.dropbox.com/s/lnbhc8yuhit4wm5/wikidata5m_alias.tar.gz?dl=1 -O wikidata5m_alias.tar.gz

--2024-04-14 19:50:10--  https://www.dropbox.com/s/6sbhm0rwo4l73jq/wikidata5m_transductive.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/6sbhm0rwo4l73jq/wikidata5m_transductive.tar.gz [following]
--2024-04-14 19:50:10--  https://www.dropbox.com/s/dl/6sbhm0rwo4l73jq/wikidata5m_transductive.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce8ae4ffcc2ae9fe8b8038b1b8a.dl.dropboxusercontent.com/cd/0/get/CRCIMxYviRyg7ATH86ofC_hUmzUzHE1viP9-87ufVl0UmWlGBgjf1it0FS8Wy3VUP7ZJ-UIFF7p3xmpxSMATOXZeeqhtPKC0w0xwIGh0Kk6v-18CtVBkXlEv6tW_THiax0ROyea-G3G4FFDwWgXYxWOq/file?dl=1# [following]
--2024-04-14 19:50:10--  https://uce8ae4ffcc2ae9fe8b8038b1b8a.dl.dropboxusercontent.com/cd/0/get/CRCIMxYviRyg7ATH86ofC_hUmzUzHE1viP9-87ufVl0UmW

In [11]:
# Extract the datasets
!tar -xzf wikidata5m_transductive.tar.gz
!gunzip wikidata5m_text.txt.gz
!tar -xzf wikidata5m_alias.tar.gz

In [12]:
!cut -f1,2 wikidata5m_entity.txt > temp.txt && mv temp.txt wikidata5m_entity.txt
!cut -f1,2 wikidata5m_relation.txt > temp.txt && mv temp.txt wikidata5m_relation.txt

In [13]:
!ls -ll

total 3298868
drwxr-xr-x 1 root   root         4096 Apr 11 13:21 sample_data
-rw-r--r-- 1 root   root    197450113 Apr 14 19:51 wikidata5m_alias.tar.gz
-rw-r--r-- 1 root   root    142927555 Apr 14 19:52 wikidata5m_entity.txt
-rw-r--r-- 1 root   root        18332 Apr 14 19:52 wikidata5m_relation.txt
-rw-r--r-- 1 root   root   2448816046 Apr 14 19:51 wikidata5m_text.txt
-rw-r--r-- 1 root   root    168258020 Apr 14 19:50 wikidata5m_transductive.tar.gz
-rw-r--r-- 1 197609 197609     104851 Nov 23  2020 wikidata5m_transductive_test.txt
-rw-r--r-- 1 197609 197609  420329153 Nov 11  2020 wikidata5m_transductive_train.txt
-rw-r--r-- 1 197609 197609     105310 Nov 11  2020 wikidata5m_transductive_valid.txt


In [14]:
!head wikidata5m_entity.txt

Q5196650	Cut Your Hair
Q912600	Straumur-Burðarás
Q47551	ditiano
Q5460288	Flora Zeta
Q1138408	URSS
Q18151267	Justice League: Battle for Metropolis
Q17981446	portal:current events/2013 december 12
Q7562391	Sonstorps IK
Q805099	euler-bernoulli law
Q4621073	2011 Afrobasket qualification


In [15]:
!head wikidata5m_relation.txt

P489	currency symbol description
P834	train depot
P2629	BBFC rating
P1677	index case of
P734	family name
P98	editor
P1001	applies to jurisdiction
P3018	located in protected area
P4545	sexually homologous with
P878	avionics


In [16]:
triplet = pd.read_csv("wikidata5m_transductive_train.txt", sep="\t", names = ["Subject", "Predicate", "Object"])
triplet

,Subject,Predicate,Object
0,Q29387131,P31,Q5
1,Q326660,P1412,Q652
2,Q7339549,P57,Q1365729
3,Q554335,P27,Q29999
4,Q20641639,P54,Q80955
...,...,...,...
20614274,Q7179406,P131,Q1439
20614275,Q5363458,P19,Q60
20614276,Q6891853,P31,Q5
20614277,Q8350496,P1142,Q564999


In [17]:
entity = pd.read_csv("wikidata5m_entity.txt", sep='\t', names = ["qID", "Name"], header=None)
entity

,qID,Name
0,Q5196650,Cut Your Hair
1,Q912600,Straumur-Burðarás
2,Q47551,ditiano
3,Q5460288,Flora Zeta
4,Q1138408,URSS
...,...,...
4809335,Q4548619,1292 in poetry
4809336,Q1435016,2005 in literature
4809337,Q3507738,4cm kanón vz. 36
4809338,Q19872232,american society of mining and reclamation


In [18]:
relationship = pd.read_csv("wikidata5m_relation.txt", sep='\t', names = ["qID", "Name"], header=None)
relationship

,qID,Name
0,P489,currency symbol description
1,P834,train depot
2,P2629,BBFC rating
3,P1677,index case of
4,P734,family name
...,...,...
820,P157,killed by
821,P1269,facet of
822,P1416,affiliation
823,P3093,recovered by


In [19]:
random_row = triplet.sample(n=1).iloc[0]


# Get names for Subject, Predicate, and Object
subject_name = entity.loc[entity['qID'] == random_row['Subject'], 'Name'].values[0]
predicate_name = relationship.loc[relationship['qID'] == random_row['Predicate'], 'Name'].values[0]
object_name = entity.loc[entity['qID'] == random_row['Object'], 'Name'].values[0]

# Get three random entities from 'entity' dataframe
random_entities = entity.sample(n=3)['Name'].values

input_model = [subject_name, predicate_name, object_name] + list(random_entities)
input_model

['peacham, vt',
 'instance of',
 'new england town',
 'scotiophyes nebrias',
 'Bunites',
 'callithomia lenea']

# Text Generation

In [ ]:
#fauno7b = FaunoModel(device = "cuda")

In [ ]:
#fauno7b.generate("Qual'è il significato della vita?")

In [ ]:
#question = "Ti forniró delle triple composte da 2 oggetti e una relazione. Genera una domanda riguarda la relazione. " + "Esempio: oggetto1 = Romolo relazione = Fondatore oggetto2 = Roma" + " Domanda: Romolo é il fondatore di Roma?" + " oggetto1 = Beaudine relazione=regista oggetto2= 'Road to Paradise'"

#prompt = f"The conversation between human and AI assistant.\n[|Human|] {question}.\n[|AI|] "
#inputs = fauno7b.tokenizer(prompt, return_tensors="pt")
#input_ids = inputs["input_ids"].cuda()
#generation_output = fauno7b.model.generate(
#            input_ids=input_ids,
#            return_dict_in_generate=True,
#            output_scores=True,
#            max_new_tokens=256
#        )
#fauno7b.tokenizer.decode(generation_output.sequences[0]).split("[|AI|]")

In [ ]:
#fauno7b.generate("Ti forniró delle triple composte da 2 oggetti e una relazione. Genera una domanda riguarda la relazione." +
#                      "Esempio: oggetto1 = Romolo relazione = Fondatore oggetto2 = Roma" +
#                      "Domanda: Romolo é il fondatore di Roma?" +
#                       "oggetto1 = Beaudine relazione=regista oggetto2= 'Road to Paradise'" )

In [20]:
model_id = "swap-uniba/LLaMAntino-2-chat-13b-hf-UltraChat-ITA"
llamantino13b = LLaMantinoModel(model_id = model_id, device = "cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [ ]:
llamantino13b.generate("Qual'è il significato della vita?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


" Non ho credenze personali o opinioni. Tuttavia, il significato della vita è un argomento complesso e soggettivo che varia da persona a persona. alcuni possono credere che il significato della vita sia trovare la felicità, raggiungere i propri obiettivi, o contribuire al mondo in qualche modo. altri possono credere che il significato della vita sia trovare un senso di scopo, di connessione, o di significato in un'esperienza spirituale o religiosa. in definitiva, il significato della vita è una questione personale che può essere plasmata da esperienze, valori e credenze individuali."

In [ ]:
llamantino13b.generate("Sei in grado di formulare delle domande?")

' Sì, come modello di lingua AI, posso formulare domande. Posso anche rispondere a domande e fornire informazioni su vari argomenti.'

In [ ]:
llamantino13b.generate("Ti forniró delle triple composte da 2 oggetti e una relazione. Genera una domanda riguarda la relazione." +
                      " Esempio: oggetto1 = Romolo relazione = Fondatore oggetto2 = Roma" +
                      " Domanda: Romolo é il fondatore di Roma?" +
                      "oggetto1 = Beaudine relazione=regista oggetto2= 'Road to Paradise'" )

" Sì, posso fornirvi una triple composta da 2 oggetti e una relazione.\n\nOggetto1 = Beaudine relazione = Regista oggetto2 = 'Road to Paradise'\n\nLa domanda è: Beaudine è il regista di 'Road to Paradise'?"

In [ ]:
llamantino13b.generate("""Ti forniró delle triple composte da un oggetto,  una relazione e quattro possibili risposte. Genera una domanda a scelta multipla.
 Esempio: oggetto1 =  libro '1984'  relazione = autore risposta1= Huxley risposta2 = Orwell risposta3 = Hemingway risposta4 = Bradbury
 Domanda: Chi ha scritto il libro '1984'?
A) Huxley
B) Orwell
C) Hemingway
D) Bradbury

 oggetto1 = 'Road to Paradise' relazione=regista risposta1= Beaudine risposta2 = Scorsese risposta3 =  Spielberg risposta4 = Nolan""")

" Domanda: Chi ha diretto il film 'Road to Paradise'?\nA) Beaudine\nB) Scorsese\nC) Spielberg\nD) Nolan"

In [ ]:
llamantino13b.generateQuestion(
    "Germania",
    "confine",
    "Irlanda",
    "Belgio",
    "Portogallo",
    "Bulgaria"
)

' Domanda: Quale paese ha una frontiera con la Germania?\nA) Irlanda\nB) Belgio\nC) Portogallo\nD) Bulgaria'

In [24]:
promptGenerator = PromptGenerator()
inputString = promptGenerator.generate(
    object_target=input_model[0],
    relationship=input_model[1],
    answer1=input_model[2],
    answer2=input_model[3],
    answer3=input_model[4],
    answer4=input_model[5]
)
inputString

"Ti forniró delle triple composte da un oggetto,  una relazione e quattro possibili risposte. Genera una domanda a scelta multipla.\nEsempio: oggetto1 =  libro '1984'  relazione = autore risposta1= Huxley risposta2 = Orwell risposta3 = Hemingway risposta4 = Bradbury\nDomanda: Chi ha scritto il libro '1984'?\nA) Huxley\nB) Orwell\nC) Hemingway\nD) Bradbury\n\n oggetto1 = peacham, vt relazione=instance of risposta1= new england town risposta2 = scotiophyes nebrias risposta3 =  Bunites risposta4 = callithomia lenea"

In [25]:
llamantino13b.generate(inputString)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


' Sì, ecco una triple composta da un oggetto, una relazione e quattro possibili risposte:\n\nOggetto1 = Peacham, VT\nRelazione = Instance of\nRisposta1 = New England town\nRisposta2 = Scots-Irish settlers\nRisposta3 = Bunites\nRisposta4 = Callithumia lenea'